In [2]:
messages = [line for line in open('SMSSpamCollection')]

In [3]:
len(messages)

5574

In [4]:
for message_no, message in enumerate(messages[:5]):
    print(message_no, message)

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

1 ham	Ok lar... Joking wif u oni...

2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

3 ham	U dun say so early hor... U c already then say...

4 ham	Nah I don't think he goes to usf, he lives around here though




### Converting into Pandas 

In [9]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [10]:
# Quoting - Remove those words whcich are not part of English
# e.g. German, French etc.
messages = pd.read_csv('SMSSpamCollection', sep = '\t', quoting = csv.QUOTE_NONE, names = ['Label', 'Message'])
messages.head(5)

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
messages.Message.values[5]

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"

In [20]:
print('Aggregate Statistics of Messages : ')
messages.groupby('Label').describe()

Aggregate Statistics of Messages : 


Message                                                               
        count unique                                                top freq
Label                                                                       
ham      4827   4518                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [22]:
messages.describe()      # Describe behaves according to Data-Types, we have here is textutal data

,Label,Message
count,5574,5574
unique,2,5171
top,ham,"Sorry, I'll call later"
freq,4827,30


### Length of Messages

In [25]:
print('Length of initial few Messages : ')
messages['Length'] = messages['Message'].map(lambda text: len(text))
messages.head()

Length of initial few Messages : 


,Label,Message,Length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [26]:
print(messages.Length.describe())

count    5574.000000
mean       80.478292
std        59.848302
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: Length, dtype: float64


In [32]:
print(list(messages.Message[messages.Length > 900]))

["For me the love should start with attraction.i should feel that I need her every time around me.she should be the first thing which comes in my thoughts.I would start the day and end it with her.she should be there every time I dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.I would cry for her.will give all my happiness and take all her sorrows.I will be ready to fight with anyone for her.I will be in love when I will be doing the craziest things for her.love will be when I don't have to proove anyone that my girl is the most beautiful lady on the whole planet.I will always be singing praises for her.love will be when I start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is with me.I would like to say a lot..will tell later.."]


### Tokenization 

In [40]:
from textblob import TextBlob

def split_into_tokens(message):
    return TextBlob(message).words

In [41]:
TextBlob('I am into Machine Learning. I\'d love to play cricket').words

WordList(['I', 'am', 'into', 'Machine', 'Learning', 'I', "'d", 'love', 'to', 'play', 'cricket'])

In [42]:
print('Tokenized messages : ')
print(messages.Message.head().apply(split_into_tokens))

Tokenized messages : 
0    [Go, until, jurong, point, crazy, Available, o...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor, U, c, already, t...
4    [Nah, I, do, n't, think, he, goes, to, usf, he...
Name: Message, dtype: object


##### Part of Speech Tags 

In [44]:
# Penn Treebank Project for Tagging of words - o/p - (word, tag)
print(TextBlob('This is for checking tags here').tags)

[('This', 'DT'), ('is', 'VBZ'), ('for', 'IN'), ('checking', 'VBG'), ('tags', 'NNS'), ('here', 'RB')]


In [45]:
"""
Verbs are the Words having tags [VB, VBD, VBG, VBN, VBP, VBZ]
Adjective are the Words having tags [JJ, JJR, JJS]
Nouns are the Words having tags [NN, NNP, NNS]
"""

'\nVerbs are the Words having tags [VB, VBD, VBG, VBN, VBP, VBZ]\nAdjective are the Words having tags [JJ, JJR, JJS]\nNouns are the Words having tags [NN, NNP, NNS]\n'

### Lemmatization 